In [7]:
import datetime as dt
import pandas as pd
import requests
import folium
import json

#print('importado')

In [8]:
boundaries_map = r'res/custom.geo.json'

In [9]:
#FUNDOS DE PREVIDÊNCIA COMO PORCENTAGEM DO PIB
df_share_gdp = pd.read_csv('res/2.1. Assets as share of GDP.csv')

#Eliminando colunas inúteis
df_share_gdp.drop('Pension Plan Type', axis=1, inplace=True)
df_share_gdp.drop('PPTYP', axis=1, inplace=True)
df_share_gdp.drop('DTYP', axis=1, inplace=True)
df_share_gdp.drop('Definition Type', axis=1, inplace=True)
df_share_gdp.drop('CTYP', axis=1, inplace=True)
df_share_gdp.drop('Contract Type', axis=1, inplace=True)
df_share_gdp.drop('VAR', axis=1, inplace=True)
df_share_gdp.drop('Variable', axis=1, inplace=True)
df_share_gdp.drop('IND', axis=1, inplace=True)
df_share_gdp.drop('Indicator', axis=1, inplace=True)
df_share_gdp.drop('Flags', axis=1, inplace=True)
df_share_gdp.drop('Flag Codes', axis=1, inplace=True)
df_share_gdp.drop('Reference Period', axis=1, inplace=True)
df_share_gdp.drop('Reference Period Code', axis=1, inplace=True)
df_share_gdp.drop('PowerCode', axis=1, inplace=True)
df_share_gdp.drop('PowerCode Code', axis=1, inplace=True)
df_share_gdp.drop('Unit', axis=1, inplace=True)
df_share_gdp.drop('YEA', axis=1, inplace=True)

#Dividindo dataframes por anos
df_share_gdp_2008=df_share_gdp[df_share_gdp['Year'] == 2008]
df_share_gdp_2008.reset_index(inplace=True, drop=True)
df_share_gdp_2009=df_share_gdp[df_share_gdp['Year'] == 2009]
df_share_gdp_2009.reset_index(inplace=True, drop=True)
df_share_gdp_2010=df_share_gdp[df_share_gdp['Year'] == 2010]
df_share_gdp_2010.reset_index(inplace=True, drop=True)
df_share_gdp_2011=df_share_gdp[df_share_gdp['Year'] == 2011]
df_share_gdp_2011.reset_index(inplace=True, drop=True)
df_share_gdp_2012=df_share_gdp[df_share_gdp['Year'] == 2012]
df_share_gdp_2012.reset_index(inplace=True, drop=True)
df_share_gdp_2013=df_share_gdp[df_share_gdp['Year'] == 2013]
df_share_gdp_2013.reset_index(inplace=True, drop=True)
df_share_gdp_2014=df_share_gdp[df_share_gdp['Year'] == 2014]
df_share_gdp_2014.reset_index(inplace=True, drop=True)
df_share_gdp_2015=df_share_gdp[df_share_gdp['Year'] == 2015]
df_share_gdp_2015.reset_index(inplace=True, drop=True)
df_share_gdp_2016=df_share_gdp[df_share_gdp['Year'] == 2016]
df_share_gdp_2016.reset_index(inplace=True, drop=True)
df_share_gdp_2017=df_share_gdp[df_share_gdp['Year'] == 2017]
df_share_gdp_2017.reset_index(inplace=True, drop=True)
df_share_gdp_2018=df_share_gdp[df_share_gdp['Year'] == 2018]
df_share_gdp_2018.reset_index(inplace=True, drop=True)

df_share_gdp_2018.head(10)

,COU,Country,Year,Unit Code,Value
0,AUS,Australia,2018,PC,137.864
1,AUT,Austria,2018,PC,5.550
2,BEL,Belgium,2018,PC,6.826
3,CAN,Canada,2018,PC,84.390
4,CZE,Czech Republic,2018,PC,9.171
5,DNK,Denmark,2018,PC,45.271
6,FIN,Finland,2018,PC,49.740
7,FRA,France,2018,PC,0.705
8,DEU,Germany,2018,PC,6.901
9,GRC,Greece,2018,PC,0.749


In [10]:
#GASTOS COM PREVIDÊNCIA COM PORCENTAGEM DO PIB
df_gdp_spending = pd.read_csv('res/1.1. OECD Pensions as share GDP.csv', header=0)

#Eliminando colunas inúteis
df_gdp_spending.drop('INDICATOR', axis=1, inplace=True)
df_gdp_spending.drop('MEASURE', axis=1, inplace=True)
df_gdp_spending.drop('FREQUENCY', axis=1, inplace=True)
df_gdp_spending.drop('Flag Codes', axis=1, inplace=True)

#Dropa linhas de gasto privado
i = 0
for gasto in df_gdp_spending['SUBJECT']:
    if gasto == 'PRIV':
        df_gdp_spending.drop([i], axis=0, inplace=True)

    i+=1
df_gdp_spending.reset_index(inplace=True,drop=True)

df_gdp_spending.head(10)

print(df_gdp_spending.columns)

Index(['LOCATION', 'SUBJECT', 'TIME', 'Value'], dtype='object')


In [11]:
#Inicializando mapa
map = folium.Map(location=[30,5], zoom_start=1.9, min_zoom=1.9, max_bounds=True)

folium.GeoJson(
    boundaries_map,
    name='fronteiras',
    style_function=lambda feature: {
        'fillColor': 'red',
        'color' : 'red',
        'weight' : 1,
        'fillOpacity' : 0.5,
        }
).add_to(map)

map.save(outfile='map.html')

map

In [126]:
#Mapa coroplético
map2 = folium.Map(location=[30,5], zoom_start=1.5, max_bounds=True)

folium.Choropleth(
    geo_data=boundaries_map,
    data=df_gdp_spending,
    columns=['LOCATION', 'Value'],
    key_on='feature.properties.LOCATION',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(map2)


ValueError: key_on `'properties.LOCATION'` not found in GeoJSON.

In [121]:
a = open('res/custom.geo.json')
a = json.load(a)
i=0

c = []

for co in a['features']:
    c.append(a['features'][i]['properties']['sov_a3'])
    print(a['features'][i]['properties']['sov_a3'])
    
    i+=1

BHS
BLZ
CAN
CRI
CUB
DOM
GTM
HND
HTI
JAM
DN1
MEX
NIC
PAN
SLV
US1
TTO
US1
ARG
BOL
CHL
COL
ECU
GB1
GUY
PER
BRA
SUR
URY
VEN
AFG
ARE
PRY
BGD
BRN
ARM
AZE
CH1
CYN
CYP
BTN
GEO
IDN
IND
IRN
IRQ
ISR
JOR
JPN
KGZ
KHM
KOR
KWT
KAZ
MMR
MNG
LAO
LBN
LKA
MYS
NPL
PAK
OMN
PRK
ISR
QAT
PHL
SAU
SYR
THA
TJK
TKM
TLS
TWN
TUR
UZB
VNM
YEM
AGO
BDI
BEN
BFA
BWA
CAF
CIV
CMR
COD
COG
DJI
DZA
EGY
ERI
ETH
GAB
GMB
GHA
GIN
GNB
GNQ
KEN
LBR
LBY
LSO
MAR
MLI
MDG
MOZ
MRT
MWI
NAM
NER
NGA
RWA
SAH
SDN
SDS
SEN
SLE
SOL
SOM
SWZ
TCD
TGO
TUN
TZA
ZAF
UGA
ZMB
ZWE
AU1
FJI
FR1
NZ1
PNG
SLB
VUT
ALB
AUT
BEL
BGR
DEU
DN1
BIH
ESP
BLR
CHE
CZE
EST
FI1
FR1
GRC
HRV
HUN
GB1
IRL
ISL
ITA
LTU
LUX
LVA
KOS
MKD
MNE
NL1
MDA
ROU
RUS
NOR
POL
PRT
SRB
SVK
SVN
SWE
UKR


In [ ]:
#Lista com todos os países do geojsonm
countr_dict = {'Country': c}
countr_dict['Country']